In [5]:
from typing import List
import math
import numpy as np
import orekit

vm = orekit.initVM()

from orekit.pyhelpers import setup_orekit_curdir, absolutedate_to_datetime
setup_orekit_curdir()

In [13]:
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.attitudes import LofOffset
from org.orekit.bodies import CelestialBodyFactory, OneAxisEllipsoid
from org.orekit.forces.drag import IsotropicDrag
from org.orekit.forces.gravity.potential import GravityFieldFactory
from org.orekit.forces.maneuvers import ImpulseManeuver
from org.orekit.frames import FramesFactory, LOFType
from org.orekit.models.earth.atmosphere import HarrisPriester
from org.orekit.orbits import CircularOrbit, OrbitType, PositionAngle
from org.orekit.propagation import PropagationType, SpacecraftState
from org.orekit.propagation.events import DateDetector, EventEnablingPredicateFilter, PythonEnablingPredicate,\
     PositionAngleDetector, PythonEventDetector, AbstractDetector, EventDetector
from org.orekit.propagation.conversion import DormandPrince853IntegratorBuilder, DSSTPropagatorBuilder
from org.orekit.propagation.semianalytical.dsst.forces import DSSTAtmosphericDrag, DSSTNewtonianAttraction, DSSTZonal
from org.orekit.time import AbsoluteDate, TimeScalesFactory
from org.orekit.utils import Constants, IERSConventions, PVCoordinatesProvider
from orekit.pyhelpers import absolutedate_to_datetime

from .inputs import Inputs
from .utils import JULIAN_DAYS_PER_YEAR, KILOMETERS, normalize_around_0_radians, normalize_around_pi, seconds_in_day

ImportError: attempted relative import with no known parent package

In [12]:
def propagate(inputs: Inputs):
    propagator = propagator_with_drag_and_J2(inputs.initial_orbit, inputs.mass)

    maneuvers = recompute_maneuvers(inputs)
    for maneuver in maneuvers:
        propagator.addEventDetector(maneuver)

    propagator.propagate(inputs.initial_orbit.getDate().shiftedBy(inputs.duration))
    ephemeris = propagator.getGeneratedEphemeris()

    times_at_each_step = [inputs.initial_orbit.getDate().shiftedBy(float(dt)) for dt in np.append(np.arange(0.0, inputs.duration, inputs.step), inputs.duration)]
    
    orbits_at_maneuvers = [CircularOrbit(ephemeris.propagate(date).getOrbit()) for date in times_at_maneuvers]
    orbits_at_each_step = [CircularOrbit(ephemeris.propagate(date).getOrbit()) for date in times_at_each_step]

    return {'dates': [absolutedate_to_datetime(orbit.getDate()) for orbit in orbits_at_maneuvers],
            'semiMajorAxis': [orbit.getA() for orbit in orbits_at_maneuvers],
            'eccentricityX': [orbit.getCircularEx() for orbit in orbits_at_maneuvers],
            'eccentricityY': [orbit.getCircularEy() for orbit in orbits_at_maneuvers],
            'inclination': [orbit.getI() for orbit in orbits_at_maneuvers],
            'raan': [orbit.getRightAscensionOfAscendingNode() for orbit in orbits_at_maneuvers]}

def new_maneuver(maneuver_pso, maneuver_approximate_date, mass, isp, orbital_period, deltaV_T, deltaV_W) -> ImpulseManeuver:
    predicate = CloseToDatePredicate(maneuver_approximate_date, orbital_period/2.)
    detector = EventEnablingPredicateFilter(PositionAngleDetector(OrbitType.CIRCULAR, PositionAngle.TRUE, maneuver_pso),
                                             predicate)
    vector_deltaV_T = Vector3D.scalarMultiply(Vector3D.PLUS_I, deltaV_T)
    vector_deltaV_W = Vector3D.scalarMultiply(Vector3D.PLUS_K, deltaV_W)
    return DateRecordingImpulseManeuver(detector, LofOffset(FramesFactory.getEME2000(), LOFType.TNW), vector_deltaV_T.add(vector_deltaV_W), isp)
    # return ImpulseManeuver(detector, LofOffset(FramesFactory.getEME2000(), LOFType.TNW), vector_deltaV_T.add(vector_deltaV_W), isp)



# drag using Harris Priester
cross_section_drag = 3. # TODO : the user must be able to set this
drag_coeff = 2.2
frame = FramesFactory.getITRF(IERSConventions.IERS_2010, False)
earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS, Constants.WGS84_EARTH_FLATTENING, frame)
atmosphere = HarrisPriester(CelestialBodyFactory.getSun(), earth)
drag_sensitive = IsotropicDrag(cross_section_drag, drag_coeff)
drag_force = DSSTAtmosphericDrag(atmosphere, drag_sensitive, Constants.WGS84_EARTH_MU)

# central body attraction
central_body = DSSTNewtonianAttraction(Constants.WGS84_EARTH_MU)

# J2 perturbation
zonal_central_body = DSSTZonal(GravityFieldFactory.getUnnormalizedProvider(6, 6))

def propagator_reference(initial_orbit, mass):
    builder = DSSTPropagatorBuilder(initial_orbit,
                                    DormandPrince853IntegratorBuilder(600.0, 86400.0, 100.0),
                                    1.0, PropagationType.MEAN, PropagationType.MEAN)
    builder.setMass(mass)
    propagator = builder.buildPropagator(builder.getSelectedNormalizedParameters())
    propagator.setEphemerisMode()
    return propagator

def propagator_with_drag_and_J2(initial_orbit, mass):
    propagator = propagator_reference(initial_orbit, mass)
    propagator.addForceModel(drag_force)
    propagator.addForceModel(central_body)
    propagator.addForceModel(zonal_central_body)
    return propagator

NameError: name 'Inputs' is not defined

In [14]:
from org.orekit.utils import JULIAN_DAYS_PER_YEAR, KILOMETERS, normalize_around_0_radians, normalize_around_pi, seconds_in_day

ImportError: cannot import name 'JULIAN_DAYS_PER_YEAR' from 'org.orekit.utils' (/home/whiskiss/anaconda3/lib/python3.8/site-packages/orekit/_orekit.cpython-38-x86_64-linux-gnu.so)